# LLM Copilot
This walkthrough covers the `functime.llm` module, which contains namespaced polars dataframe methods to interoperate Large Language Models (LLMs) with functime.

Let's use OpenAI's GPT models to analyze commodity price forecasts created by a functime forecaster. By default we use `gpt-3.5-turbo`.

### Load data

In [ ]:
import os

from IPython.display import Markdown, display

os.environ["OPENAI_API_KEY"] = "..."  # Your API key here

In [ ]:
%%capture
import polars as pl

from functime.cross_validation import train_test_split
from functime.forecasting import knn

In [ ]:
y = pl.read_parquet("../../data/commodities.parquet")
entity_col, time_col, target_col = y.columns
test_size = 30
freq = "1mo"
y_train, y_test = train_test_split(test_size)(y)
print("🎯 Target variable (y) -- train set:")
y_train.collect()

We'll make a prediction using a knn forecaster.

In [ ]:
# Univariate time-series fit with automated lags
forecaster = knn(freq="1mo", lags=24)
forecaster.fit(y=y_train)
y_pred = forecaster.predict(fh=test_size)
y_pred.head()

We'll also provide a short description of the dataset to aid the LLM in its analysis.

In [ ]:
dataset_context = (
    "This dataset comprises of forecasted commodity prices between 2020 to 2023."
)

### Analyze Forecasts

Let's take a look at aluminum and European banana prices. You can select multiple (or just one) entity / time-series to analyze through the `basket` variable.

In [ ]:
analysis = y_pred.llm.analyze(
    context="This dataset comprises of forecasted commodity prices between 2020 to 2023.",
    basket=["Aluminum", "Banana, Europe"],
)
display(Markdown(analysis))

### Compare Forecasts

Let's now compare the previous selection with a new one. We'll refer to these as baskets A and B.

In [ ]:
basket_a = ["Aluminum", "Banana, Europe"]
basket_b = ["Chicken", "Cocoa"]

Now compare!

In [ ]:
comparison = y_pred.llm.compare(basket=basket_a, other_basket=basket_b)
display(Markdown(comparison))